In [1]:
import pandas
import os
import numpy as np

data = pandas.read_csv('features_SPY.csv', delimiter=',')
data['Date'] = pandas.to_datetime(data['Date'])

X = data.iloc[:, 1:23]
#print(X)

y = np.where(data['Return_2'] >= 0, 1.0, 0.0)
print(y[:20])
y = np.roll(y, -2) # yes it's future returns, but last values in array - are random trash!
print(y.shape)

[1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0.]
(1510,)


In [2]:
days = [0, 1, 2, 3, 5, 10, 20, 40, 60, 120, 250]

#preparing features
def getFeatures(X, days, period):
    index_day = days.index(period)
    featureCount = (1 + days[index_day - 1]) * 2 + 2;
    
    
    #print("ii",index_day)
    return_key = "Return_" + str(period)
    volume_key = "AverageVolume_" + str(period)
    print(return_key, volume_key)

    array = np.empty([0, featureCount])
    for index, row in X.iterrows():
        #if (index > 2): break;
        features = np.empty(0)
        #print(row)
        column = X[return_key]
        #print(column[index])
        #print("i",index)
        #print("a", features)
        features = np.append(features, column[index])
        #print("aa", features)
        for lag in range(1, days[index_day - 1] + 1):
            #print("lag = %d" % lag)
            #numpy.append(features, value
            if (index - lag >= 0):
                features = np.append(features, column[index - lag])
            else:
                features = np.append(features, float("NaN"))
        
        column = X[volume_key]
        features = np.append(features, column[index])
        for lag in range(1, days[index_day - 1] + 1):
            #print("lag = %d" % lag)
            #numpy.append(features, value
            if (index - lag >= 0):
                features = np.append(features, column[index - lag])
            else:
                features = np.append(features, float("NaN"))

        column = X['DayOfWeek']
        features = np.append(features, column[index])
        column = X['DayOfMonth']
        features = np.append(features, column[index])
        
        #print(features)
        #https://stackoverflow.com/a/30305148
        #print(features[None, :].shape)
        #print(array.shape)
        array = np.vstack([array, features[None, :]])
        #print("array",array)
    print('\n')
    
    return array

print(getFeatures(X, days, 2).shape)

Return_2 AverageVolume_2


(1510, 6)


In [3]:
Xfeatures = getFeatures(X, days, 2)

features_map = {"Feature" + str(i): Xfeatures[:,i] for i in range(0, Xfeatures[0].size)}
#print(aa["column0"].shape)
Xfeatures = pandas.DataFrame(features_map)
print("index = ", Xfeatures.index)
print(type(Xfeatures))
#XFeatures = pandas.Datafame(XFeatures)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Xfeatures, y, 
                                      test_size=0.3, 
                                      random_state=241)


X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)

#print(X_train.shape)
#print(X_test.shape)
print(X_train)

Return_2 AverageVolume_2


index =  RangeIndex(start=0, stop=1510, step=1)
<class 'pandas.core.frame.DataFrame'>
      Feature0  Feature1     Feature2     Feature3  Feature4  Feature5
0     0.011428 -0.014834  256726950.0  224754800.0       2.0      17.0
1    -0.032661 -0.035755  270323850.0  255051550.0       1.0       6.0
2     0.004041  0.011213  208322300.0  202053000.0       2.0       7.0
3    -0.013739 -0.016527  158817450.0  155611900.0       1.0      23.0
4    -0.007770 -0.007860  168925200.0  129434850.0       3.0       2.0
...        ...       ...          ...          ...       ...       ...
1052 -0.009724 -0.023326  257489150.0  228480000.0       4.0      29.0
1053  0.004654  0.015897  166809100.0  185718600.0       4.0      20.0
1054  0.017879  0.010813  105826300.0  156848150.0       4.0      23.0
1055  0.010298  0.012168  136325450.0  131790850.0       4.0      18.0
1056  0.018623  0.006132  156555550.0   88200450.0       3.0      13.0

[1057 rows x 6 columns]


In [4]:
from sklearn.preprocessing import StandardScaler

#https://stackoverflow.com/questions/35723472/how-to-use-sklearn-fit-transform-with-pandas-and-return-dataframe-instead-of-num
def scaleDataFrame(dataframe, debugPrint = False):
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(dataframe)
    dataframe = pandas.DataFrame(scaled_features, index=dataframe.index, columns=dataframe.columns)
    
    #check
    if debugPrint == True:
        for column in dataframe:
            print("[%s] mean = %.4f, variance = %.4f" % 
                  (column, dataframe[column].mean(), dataframe[column].var()))
    
    return dataframe

In [5]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

from sklearn.model_selection import cross_val_score

from datetime import datetime

# C = 1000.0 algorithm hangs!
C = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
#C = {1.0}
#grid = {'C': np.power(10.0, np.arange(-4, 0))}

for penalty in C:
    print("trying C = " + str(penalty))
    kf = KFold(n_splits=3, shuffle=False, random_state=241)
    
    X_train = scaleDataFrame(X_train, debugPrint = False)

    clf = svm.SVC(C = penalty, kernel = 'linear', random_state=241)
    N = 300
    XX = X_train.iloc[:N, :]
    yy = y_train[:N]
    print("from", datetime.now())
    scores = cross_val_score(clf, XX, yy, scoring='accuracy', cv=kf)
    print(scores)
    print(scores.mean())
    print("to", datetime.now())
    
    continue;
    for train_index, test_index in cv.split(X_train):
        #clf = svm.SVC(kernel='linear', random_state=241)
        clf = svm.SVC(C = 1.0, kernel = 'linear', verbose = True, random_state=241)
        #XX = X_train.loc[train_index, ['Return_1', 'AverageVolume_1']]
        XX = X_train.loc[train_index, :]
        yy = y_train[train_index]
        #print(train_index[:])
        #print(X_train[0:13])
        #print(XX.iloc[:20,:])
        print(XX.iloc[:20, 0:1])
        #print(yy[:10])
        
        print("XX type",type(XX))
        print(XX.shape)
        print(yy.shape)
        #XX.to_csv("XX.csv", sep = ";")
        print("fit start")
        clf.fit(XX.iloc[:100, 0:2], yy[:100])
        print("fit end")
        y_pred = clf.predict(X_train.iloc[:,0:2])
        print(y_pred)
        
        #clf.fit(X_train[train_index], y_train[train_index])
#gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv)
#gs.fit(X, y)

trying C = 0.0001
from 2019-11-13 16:17:40.377111
[0.58 0.55 0.61]
0.58
to 2019-11-13 16:17:40.407110
trying C = 0.001
from 2019-11-13 16:17:40.416110
[0.58 0.55 0.61]
0.58
to 2019-11-13 16:17:40.450136
trying C = 0.01
from 2019-11-13 16:17:40.455110
[0.58 0.55 0.61]
0.58
to 2019-11-13 16:17:40.481133
trying C = 0.1
from 2019-11-13 16:17:40.486195
[0.58 0.55 0.61]
0.58
to 2019-11-13 16:17:40.514111
trying C = 1.0
from 2019-11-13 16:17:40.519214
[0.58 0.55 0.61]
0.58
to 2019-11-13 16:17:40.547112
trying C = 10.0
from 2019-11-13 16:17:40.554111
[0.58 0.55 0.61]
0.58
to 2019-11-13 16:17:40.605111
trying C = 100.0
from 2019-11-13 16:17:40.612129
[0.58 0.55 0.61]
0.58
to 2019-11-13 16:17:40.783114


In [6]:
import import_ipynb
import FeatureStore

FeatureStore.getTrainDataset(2)
FeatureStore.getTarget(period=2)

importing Jupyter notebook from FeatureStore.ipynb
[0. 1. 0. ... 0. 0. 1.]




array([1., 1., 0., ..., 0., 1., 1.])